___
___
# `Код`

`Перезапуск`

In [2]:
# 1 - отправка в Kaggle
# 2 - локальный тест на 5-ти предпоследних неделях
# 3 - локальный тест на 5-ти последних неделях декабря 2018 года

submit = 1

## `1. Загрузка данных`

In [3]:
import numpy as np
import pandas as pd

from copy import deepcopy
import itertools

from tqdm import tqdm
from matplotlib import pyplot as plt

product = pd.read_table('hse-dab-naf-22/PRODUCT.csv', sep=';')
location = pd.read_table('hse-dab-naf-22/LOCATION.csv', sep=';')

data = pd.read_table('hse-dab-naf-22/train.csv', sep=',')
sample_submission = pd.read_table('hse-dab-naf-22/sample_submission.csv', sep=',')

___
## `2. Маски для разделение данных на обучающую и тестовую выборки`

In [4]:
data['period_dt'] = pd.to_datetime(data['period_dt'])

### `2.1. Маска для сабмита в Kaggle`

In [5]:
def split_data_kaggle(data):
    ids_kaggle = np.array(sample_submission['id'].values)
    mask = data['id'].isin(ids_kaggle)
    
    train_mask = ~mask
    test_mask = mask
    
    return train_mask, test_mask

### `2.2. Маска для предсказания дат [04.11.2019, 11.11.2019, 18.11.2019, 25.11.2019, 02.12.2019]`

In [6]:
def split_data_5_prev_weeks(data):
    mask_first = \
        (
            (data['period_dt'].dt.day == 4) | \
            (data['period_dt'].dt.day == 11) | \
            (data['period_dt'].dt.day == 18) | \
            (data['period_dt'].dt.day == 25)
        ) & \
        (data['period_dt'].dt.month == 11) & \
        (data['period_dt'].dt.year == 2019)

    mask_second = (data['period_dt'].dt.day == 2) & (data['period_dt'].dt.month == 12) & (data['period_dt'].dt.year == 2019)
    mask = mask_first | mask_second
    
    # Вырезаем все те даты, которые идут после упомянутых в скобках
    mask_remove = (data['period_dt'].dt.day > 2) & (data['period_dt'].dt.month == 12) & (data['period_dt'].dt.year == 2019)
    
    train_mask = ~mask_remove & ~mask
    test_mask = ~mask_remove & mask
    
    return train_mask, test_mask

### `2.3. Маска для предсказания дат [03.12.2018, 10.12.2018, 17.12.2018, 24.12.2018, 31.12.2018]`

In [7]:
def split_data_5_last_weeks_of_2018(data):
    mask = \
        (
            (data['period_dt'].dt.day == 3) | \
            (data['period_dt'].dt.day == 10) | \
            (data['period_dt'].dt.day == 17) | \
            (data['period_dt'].dt.day == 24) | \
            (data['period_dt'].dt.day == 31)
        ) & \
        (data['period_dt'].dt.month == 12) & \
        (data['period_dt'].dt.year == 2018)
    
    # Вырезаем все те даты, которые идут после упомянутых в скобках
    mask_remove = (data['period_dt'].dt.year == 2019)

    train_mask = ~mask_remove & ~mask
    test_mask = ~mask_remove & mask
    
    return train_mask, test_mask

___
## `3. Предобработка данных`

In [8]:
features_cat = ['period_dt', 'location_id', 'product_id', 'PROMO1_FLAG', 'PROMO2_FLAG', 'AUTORIZATION_FLAG']
features_num = ['PRICE_REGULAR', 'PRICE_AFTER_DISC', 'NUM_CONSULTANT']
target = 'demand'

### `3.1. Заполнение пропусков в некоторых признаках`

In [9]:
def fill_nans(data):
    mask = pd.isna(data['PROMO1_FLAG'])
    data.loc[mask, 'PROMO1_FLAG'] = 0.0

    mask = pd.isna(data['PROMO2_FLAG'])
    data.loc[mask, 'PROMO2_FLAG'] = 0.0

    mask = pd.isna(data['PRICE_REGULAR'])
    data.loc[mask, 'PRICE_REGULAR'] = data['PRICE_REGULAR'].mean()

    mask = pd.isna(data['PRICE_AFTER_DISC'])
    data.loc[mask, 'PRICE_AFTER_DISC'] = data['PRICE_AFTER_DISC'].mean()

    mask = pd.isna(data['NUM_CONSULTANT'])
    data.loc[mask, 'NUM_CONSULTANT'] = 0.0

    mask = pd.isna(data['AUTORIZATION_FLAG'])
    data.loc[mask, 'AUTORIZATION_FLAG'] = 1.0
    
    return data

### `3.2. Создание новых признаков в виде дня, месяца и года`

In [10]:
def day_month_year_features(data, features_cat, features_num):
    data['period_dt'] = pd.to_datetime(data['period_dt'])
    
    data['day'] = data['period_dt'].dt.day
    data['month'] = data['period_dt'].dt.month
    data['year'] = data['period_dt'].dt.year
    
    features_num += ['day', 'month', 'year']
    
    return data

### `3.3. Удаление признаков`

In [11]:
def delete_feature(data, features_cat, features_num, feature_names):
    for feature_name in feature_names:
        data = data.drop(columns=[feature_name])

        if feature_name in features_num:
            features_num.remove(feature_name)

        if feature_name in features_cat:
            features_cat.remove(feature_name)
        
    return data

### `3.4. Таблица PRODUCT`

In [12]:
def product_table_features(data, features_cat, features_num, product, is_useful_columns=True):
    table = product.copy()

    mask = ~pd.isna(data['product_id'])
    elements_unique = data[mask]['product_id'].unique()

    table_sliced = table.loc[table['PRODUCT_RK'].isin(elements_unique)].copy()    
    
    useful_columns = []
    if is_useful_columns:
        for col in table_sliced:
            #---------------------------------------------------------------#
            unique_count = table_sliced[col].unique().size
    
            if (unique_count > 1) and (unique_count != elements_unique.size):
                if ('RK' in col) or ('ATTRIB' in col):
                    useful_columns += [col]
            #---------------------------------------------------------------#

        table_sliced = table_sliced[['PRODUCT_RK'] + useful_columns]

    features_count = table_sliced.shape[1] - 1
    features_names = table_sliced.keys()[1:]

    data[features_names] = ''

    for elem in tqdm(elements_unique):
        mask_1 = (table_sliced['PRODUCT_RK'] == elem)
        mask_2 = (data['product_id'] == elem)
        data.loc[mask_2, features_names] = table_sliced[features_names][mask_1].values

    features_cat += list(features_names)

    return data

### `3.5. Таблица LOCATION`

In [13]:
def location_table_features(data, features_cat, features_num, location, is_useful_columns=True):
    table = location.copy()

    mask = ~pd.isna(data['location_id'])
    elements_unique = data[mask]['location_id'].unique()

    table_sliced = table.loc[table['STORE_LOCATION_RK'].isin(elements_unique)].copy()    
    
    useful_columns = []
    if is_useful_columns:
        for col in table_sliced:
            #---------------------------------------------------------------#
            unique_count = table_sliced[col].unique().size
    
            if (unique_count > 1) and (unique_count != elements_unique.size):
                if ('RK4' in col) or ('ATTRIB2' in col) or ('ATTRIB3' in col) or \
                ('ATTRIB4' in col) or ('ATTRIB8' in col) or \
                ('ATTRIB9' in col) or ('ATTRIB10' in col):
                    useful_columns += [col]
            #---------------------------------------------------------------#

        table_sliced = table_sliced[['STORE_LOCATION_RK'] + useful_columns]

    features_count = table_sliced.shape[1] - 1
    features_names = table_sliced.keys()[1:]

    data[features_names] = ''

    for elem in tqdm(elements_unique):
        mask_1 = (table_sliced['STORE_LOCATION_RK'] == elem)
        mask_2 = (data['location_id'] == elem)
        data.loc[mask_2, features_names] = table_sliced[features_names][mask_1].values

    features_cat += list(features_names)

    return data

### `3.6.  lag-features`

In [14]:
def lagged_features(
    data,
    cat,
    num,
    target_var = 'demand',
    org_id_columns = ['product_id', 'location_id'],
    all_id_columns = ['product_id', 'location_id', 'period_dt'],
    lags = [5],
    windows = [5],
    aggregation_methods = {'mean', 'median', 'var', 'std'},
    filters = None):
    
    out_df = deepcopy(data)
    
    have_filters = True
    
    if filters is None:
        have_filters = False
        filters = {'':{''}}
        
    keys, values = zip(*filters.items())
    
    #_____________________________#
    
    for bundle in itertools.product(*values):
        condition = ''
        
        if have_filters:
            condition = ' & '.join([keys[i] + filters[keys[i]][bundle[i]] for i in range(len(keys))])
            
        name =  '_'.join([bundle[i] for i in range(len(keys))])
        
        if len(condition) > 0:
            _idx = data.eval(condition)
        else:
            _idx = data.index >= 0

        if len(data[_idx]) > 0:

            #_______________#
            
            for w in tqdm(windows):

                lf_df = data[_idx].set_index(all_id_columns)[target_var].\
                    groupby(level=org_id_columns).\
                        apply(lambda x: x.rolling(window=w, min_periods = 1).\
                            agg(aggregation_methods))

                #____________#
                
                for l in tqdm(lags):

                    if have_filters:
                        new_names = \
                            {x: "lag{0}_wdw{1}_{2}_{3}".format(l, w, x, name) for x in lf_df.columns }
                    else:
                        new_names = \
                            {x: "lag{0}_wdw{1}_{2}".format(l, w, x) for x in lf_df.columns }

                    out_df = \
                        pd.merge(
                            out_df,
                            lf_df.shift(l).reset_index().rename(columns = new_names),
                            how='left',
                            on=all_id_columns
                        )
    #___________#
    
    new_numeric_features = []

    for col in out_df.columns:
        if col[:3] == 'lag':
            new_numeric_features += [col]
            
    num += new_numeric_features
    
    assert (out_df[num + cat].shape[1] + 1) == (out_df.shape[1])

    return out_df

___
## `4. Разделение на обучающую и тестовую выборки`

In [15]:
train_mask_kaggle, test_mask_kaggle = split_data_kaggle(data)
train_mask_5_prev_weeks, test_mask_5_prev_weeks = split_data_5_prev_weeks(data)
train_mask_5_last_weeks_of_2018, test_mask_5_last_weeks_of_2018 = split_data_5_last_weeks_of_2018(data)

`Перезапуск`

In [16]:
if submit == 1:
    train_mask, test_mask = train_mask_kaggle, test_mask_kaggle
elif submit == 2:
    train_mask, test_mask = train_mask_5_prev_weeks, test_mask_5_prev_weeks
elif submit == 3:
    train_mask, test_mask = train_mask_5_last_weeks_of_2018, test_mask_5_last_weeks_of_2018
else:
    print('clown')

___
### `*. Формируем данные`

`Удаляем 'id'`

In [16]:
data = delete_feature(data, features_cat, features_num, ['id'])

`Формируем lag-features`

In [17]:
#mask_location = ~pd.isna(data['location_id'])
#mask_product = ~pd.isna(data['product_id'])

In [18]:
#data = data[mask_location & mask_product]

In [19]:
#data = \
#    lagged_features(
#        data,
#        features_cat,
#        features_num,
#        lags = [0, 26, 52, 78, 104, 130],
#        windows = [12, 26, 52],
#        filters = None)

`Добавляем новые признаки в виде дня, месяца и года`

In [20]:
data = day_month_year_features(data, features_cat, features_num)

`Удаляем признак 'period_dt'`

In [21]:
data = delete_feature(data, features_cat, features_num, ['period_dt'])

`Заполняем пропуски`

In [22]:
data = fill_nans(data)

`Добавление признаков из таблицы LOCATION`

In [23]:
#data = location_table_features(data, features_cat, features_num, location, is_useful_columns=True)

`Добавление признаков из таблицы PRODUCT`

In [24]:
data = product_table_features(data, features_cat, features_num, product, is_useful_columns=True)

100%|██████████████████████████████████████████████████████████████████████████████| 2140/2140 [22:09<00:00,  1.61it/s]


___
### `*. Разделение`

`Перезапуск`

In [26]:
#train = data[train_mask[mask_location & mask_product]].copy()
#test = data[test_mask[mask_location & mask_product]].copy()

train = data[train_mask].copy()
test = data[test_mask].copy()

`Обрежем тестовую выборку так, чтобы ею можно было бы валидировать результаты. Имеем смысл только для локального тестирования.`

In [691]:
if submit == 2 or submit == 3:
    test = test[~pd.isna(test['demand'])]

In [692]:
test.head()

,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,day,...,PRODUCT_ATTRIB10_hashing,PRODUCT_ATTRIB12_hashing,PRODUCT_ATTRIB13_hashing,PRODUCT_ATTRIB15_hashing,PRODUCT_ATTRIB16_hashing,PRODUCT_ATTRIB18_hashing,PRODUCT_ATTRIB21_hashing,PRODUCT_ATTRIB22_hashing,PRODUCT_ATTRIB23_hashing,PRODUCT_ATTRIB25_hashing
10,1162.0,22939.0,NaN,0.0,0.0,3649.0,2189.40,0.0,1.0,30,...,469EDF2953F2FE3282C01BB4DA50D3DE,469EDF2953F2FE3282C01BB4DA50D3DE,082C6C1C596475C2DBA0649E9C3EF4CC,B2581C791DBB8894BFDCFB66B60A17ED,ABA19183688552984DCF8FE467B5D008,0493DC077A6D8708F386CC48B88E1EF3,8CE9C7F761F769882B2C8D05B153760B,EA4B7C50D9AEF0EA71897172C02442B8,082C6C1C596475C2DBA0649E9C3EF4CC,E054D49EAC1E947B5CB0EE77952C5F9E
7781,1162.0,23162.0,NaN,0.0,0.0,3799.0,2279.40,0.0,1.0,30,...,469EDF2953F2FE3282C01BB4DA50D3DE,469EDF2953F2FE3282C01BB4DA50D3DE,082C6C1C596475C2DBA0649E9C3EF4CC,B2581C791DBB8894BFDCFB66B60A17ED,ABA19183688552984DCF8FE467B5D008,6022A682C75D6B2834935B68C518BC04,8CE9C7F761F769882B2C8D05B153760B,EA4B7C50D9AEF0EA71897172C02442B8,082C6C1C596475C2DBA0649E9C3EF4CC,8CF5176515BEEC570D4FB22350A55641
8574,1162.0,23163.0,NaN,0.0,0.0,3799.0,2849.25,0.0,1.0,16,...,469EDF2953F2FE3282C01BB4DA50D3DE,469EDF2953F2FE3282C01BB4DA50D3DE,082C6C1C596475C2DBA0649E9C3EF4CC,B2581C791DBB8894BFDCFB66B60A17ED,ABA19183688552984DCF8FE467B5D008,6022A682C75D6B2834935B68C518BC04,8CE9C7F761F769882B2C8D05B153760B,EA4B7C50D9AEF0EA71897172C02442B8,082C6C1C596475C2DBA0649E9C3EF4CC,8CF5176515BEEC570D4FB22350A55641
14686,798.0,23632.0,NaN,0.0,0.0,3999.0,2399.40,0.0,1.0,23,...,469EDF2953F2FE3282C01BB4DA50D3DE,469EDF2953F2FE3282C01BB4DA50D3DE,082C6C1C596475C2DBA0649E9C3EF4CC,B2581C791DBB8894BFDCFB66B60A17ED,ABA19183688552984DCF8FE467B5D008,0493DC077A6D8708F386CC48B88E1EF3,D165570588A15A2EB9A9E24D2E8E2FBD,EA4B7C50D9AEF0EA71897172C02442B8,082C6C1C596475C2DBA0649E9C3EF4CC,8CF5176515BEEC570D4FB22350A55641
14797,1347.0,23633.0,NaN,0.0,0.0,3999.0,2999.25,0.0,1.0,30,...,469EDF2953F2FE3282C01BB4DA50D3DE,469EDF2953F2FE3282C01BB4DA50D3DE,082C6C1C596475C2DBA0649E9C3EF4CC,B2581C791DBB8894BFDCFB66B60A17ED,ABA19183688552984DCF8FE467B5D008,0493DC077A6D8708F386CC48B88E1EF3,8CE9C7F761F769882B2C8D05B153760B,EA4B7C50D9AEF0EA71897172C02442B8,082C6C1C596475C2DBA0649E9C3EF4CC,8CF5176515BEEC570D4FB22350A55641


` Обучающую выборку будем иметь в двух видах:`
- `train_full - это весь train, в котором не факт, что везде известна целевая переменная`
- `train - это срез исходного train'а по объектам с известными целевыми переменными`

In [693]:
train_full = train.copy()
train = train[~pd.isna(train['demand'])].copy()

In [694]:
train_full.shape, train.shape, test.shape

((2332510, 31), (274471, 31), (593, 31))

___
## `5. Запуск модели`

In [695]:
x_train = train.drop(columns=['demand']).copy()
y_train = train['demand'].copy()

x_test = test.drop(columns=['demand']).copy()
y_test = test['demand'].copy()

In [696]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((274471, 30), (274471,), (593, 30), (593,))

In [697]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [698]:
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
import xgboost as xgb
import lightgbm as ltb

In [699]:
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error

In [700]:
column_transformer = ColumnTransformer([
        ('scaler', StandardScaler(), features_num),
        ('ohe', OneHotEncoder(), features_cat)
    ], remainder='passthrough')

column_transformer.fit(pd.concat([x_train, x_test], axis=0))

ColumnTransformer(remainder='passthrough',
                  transformers=[('scaler', StandardScaler(),
                                 ['PRICE_REGULAR', 'PRICE_AFTER_DISC',
                                  'NUM_CONSULTANT', 'day', 'month', 'year']),
                                ('ohe', OneHotEncoder(),
                                 ['location_id', 'product_id', 'PROMO1_FLAG',
                                  'PROMO2_FLAG', 'AUTORIZATION_FLAG',
                                  'PRODUCT_LVL_RK6', 'PRODUCT_ATTRIB2_hashing',
                                  'PRODUCT_ATTRIB3_hashing',
                                  'PRODUCT_ATTRIB4_hashi...
                                  'PRODUCT_ATTRIB7_hashing',
                                  'PRODUCT_ATTRIB8_hashing',
                                  'PRODUCT_ATTRIB9_hashing',
                                  'PRODUCT_ATTRIB10_hashing',
                                  'PRODUCT_ATTRIB12_hashing',
                                  

___

In [701]:
y_scaler = StandardScaler()
y_scaler.fit(np.array(y_train).reshape(-1, 1))

StandardScaler()

In [702]:
y_train_scaled = y_scaler.transform(np.array(y_train).reshape(-1, 1))

In [743]:
model = CatBoostRegressor(
    verbose=True,
    depth=5,
    learning_rate=0.099,
    l2_leaf_reg=5
)

model.fit(column_transformer.transform(x_train), y_train)

0:	learn: 0.7128356	total: 30.3ms	remaining: 30.2s
1:	learn: 0.7019605	total: 60.1ms	remaining: 30s
2:	learn: 0.6930821	total: 91.2ms	remaining: 30.3s
3:	learn: 0.6848165	total: 119ms	remaining: 29.7s
4:	learn: 0.6778952	total: 151ms	remaining: 30s
5:	learn: 0.6721427	total: 181ms	remaining: 29.9s
6:	learn: 0.6667531	total: 210ms	remaining: 29.8s
7:	learn: 0.6620622	total: 237ms	remaining: 29.3s
8:	learn: 0.6579555	total: 268ms	remaining: 29.5s
9:	learn: 0.6544402	total: 296ms	remaining: 29.3s
10:	learn: 0.6509436	total: 324ms	remaining: 29.2s
11:	learn: 0.6477340	total: 350ms	remaining: 28.8s
12:	learn: 0.6452354	total: 377ms	remaining: 28.6s
13:	learn: 0.6428369	total: 403ms	remaining: 28.4s
14:	learn: 0.6405069	total: 432ms	remaining: 28.3s
15:	learn: 0.6383383	total: 457ms	remaining: 28.1s
16:	learn: 0.6359942	total: 495ms	remaining: 28.6s
17:	learn: 0.6344909	total: 528ms	remaining: 28.8s
18:	learn: 0.6328019	total: 556ms	remaining: 28.7s
19:	learn: 0.6311662	total: 586ms	remainin

164:	learn: 0.5705240	total: 4.63s	remaining: 23.4s
165:	learn: 0.5702950	total: 4.69s	remaining: 23.6s
166:	learn: 0.5700952	total: 4.72s	remaining: 23.5s
167:	learn: 0.5698992	total: 4.74s	remaining: 23.5s
168:	learn: 0.5697453	total: 4.77s	remaining: 23.5s
169:	learn: 0.5695413	total: 4.8s	remaining: 23.4s
170:	learn: 0.5692386	total: 4.82s	remaining: 23.4s
171:	learn: 0.5690730	total: 4.85s	remaining: 23.3s
172:	learn: 0.5688869	total: 4.87s	remaining: 23.3s
173:	learn: 0.5687221	total: 4.9s	remaining: 23.3s
174:	learn: 0.5685655	total: 4.93s	remaining: 23.2s
175:	learn: 0.5684117	total: 4.96s	remaining: 23.2s
176:	learn: 0.5682688	total: 4.98s	remaining: 23.2s
177:	learn: 0.5681272	total: 5.01s	remaining: 23.1s
178:	learn: 0.5679907	total: 5.04s	remaining: 23.1s
179:	learn: 0.5677949	total: 5.07s	remaining: 23.1s
180:	learn: 0.5676230	total: 5.09s	remaining: 23s
181:	learn: 0.5674079	total: 5.12s	remaining: 23s
182:	learn: 0.5672570	total: 5.14s	remaining: 23s
183:	learn: 0.567168

325:	learn: 0.5489592	total: 9.25s	remaining: 19.1s
326:	learn: 0.5488464	total: 9.28s	remaining: 19.1s
327:	learn: 0.5487488	total: 9.31s	remaining: 19.1s
328:	learn: 0.5486921	total: 9.33s	remaining: 19s
329:	learn: 0.5485960	total: 9.36s	remaining: 19s
330:	learn: 0.5484908	total: 9.38s	remaining: 19s
331:	learn: 0.5484056	total: 9.41s	remaining: 18.9s
332:	learn: 0.5482376	total: 9.48s	remaining: 19s
333:	learn: 0.5481176	total: 9.54s	remaining: 19s
334:	learn: 0.5480149	total: 9.65s	remaining: 19.2s
335:	learn: 0.5479202	total: 9.71s	remaining: 19.2s
336:	learn: 0.5477515	total: 9.74s	remaining: 19.2s
337:	learn: 0.5475845	total: 9.77s	remaining: 19.1s
338:	learn: 0.5475388	total: 9.8s	remaining: 19.1s
339:	learn: 0.5474662	total: 9.84s	remaining: 19.1s
340:	learn: 0.5474038	total: 9.86s	remaining: 19.1s
341:	learn: 0.5473307	total: 9.89s	remaining: 19s
342:	learn: 0.5471307	total: 9.93s	remaining: 19s
343:	learn: 0.5470271	total: 9.96s	remaining: 19s
344:	learn: 0.5469342	total: 

489:	learn: 0.5355200	total: 14.7s	remaining: 15.3s
490:	learn: 0.5354195	total: 14.7s	remaining: 15.3s
491:	learn: 0.5353672	total: 14.7s	remaining: 15.2s
492:	learn: 0.5352585	total: 14.8s	remaining: 15.2s
493:	learn: 0.5351724	total: 14.8s	remaining: 15.2s
494:	learn: 0.5351147	total: 14.8s	remaining: 15.1s
495:	learn: 0.5350281	total: 14.9s	remaining: 15.1s
496:	learn: 0.5349000	total: 14.9s	remaining: 15.1s
497:	learn: 0.5348606	total: 14.9s	remaining: 15s
498:	learn: 0.5347803	total: 15s	remaining: 15s
499:	learn: 0.5347622	total: 15s	remaining: 15s
500:	learn: 0.5347117	total: 15s	remaining: 14.9s
501:	learn: 0.5346723	total: 15s	remaining: 14.9s
502:	learn: 0.5345779	total: 15.1s	remaining: 14.9s
503:	learn: 0.5345477	total: 15.1s	remaining: 14.9s
504:	learn: 0.5344460	total: 15.1s	remaining: 14.8s
505:	learn: 0.5343737	total: 15.1s	remaining: 14.8s
506:	learn: 0.5343387	total: 15.2s	remaining: 14.8s
507:	learn: 0.5343322	total: 15.2s	remaining: 14.7s
508:	learn: 0.5342806	tota

651:	learn: 0.5256758	total: 20s	remaining: 10.7s
652:	learn: 0.5256383	total: 20s	remaining: 10.6s
653:	learn: 0.5255771	total: 20.1s	remaining: 10.6s
654:	learn: 0.5255410	total: 20.1s	remaining: 10.6s
655:	learn: 0.5255072	total: 20.1s	remaining: 10.5s
656:	learn: 0.5254461	total: 20.1s	remaining: 10.5s
657:	learn: 0.5254272	total: 20.2s	remaining: 10.5s
658:	learn: 0.5253845	total: 20.2s	remaining: 10.4s
659:	learn: 0.5253089	total: 20.2s	remaining: 10.4s
660:	learn: 0.5252756	total: 20.2s	remaining: 10.4s
661:	learn: 0.5252122	total: 20.3s	remaining: 10.4s
662:	learn: 0.5251819	total: 20.3s	remaining: 10.3s
663:	learn: 0.5251181	total: 20.3s	remaining: 10.3s
664:	learn: 0.5250453	total: 20.4s	remaining: 10.3s
665:	learn: 0.5249396	total: 20.4s	remaining: 10.2s
666:	learn: 0.5248589	total: 20.4s	remaining: 10.2s
667:	learn: 0.5247926	total: 20.5s	remaining: 10.2s
668:	learn: 0.5247598	total: 20.5s	remaining: 10.1s
669:	learn: 0.5247339	total: 20.5s	remaining: 10.1s
670:	learn: 0.52

815:	learn: 0.5179483	total: 25.8s	remaining: 5.81s
816:	learn: 0.5178744	total: 25.8s	remaining: 5.78s
817:	learn: 0.5177893	total: 25.9s	remaining: 5.75s
818:	learn: 0.5177324	total: 25.9s	remaining: 5.72s
819:	learn: 0.5176694	total: 25.9s	remaining: 5.69s
820:	learn: 0.5176476	total: 26s	remaining: 5.66s
821:	learn: 0.5175859	total: 26s	remaining: 5.64s
822:	learn: 0.5175696	total: 26.1s	remaining: 5.61s
823:	learn: 0.5175436	total: 26.1s	remaining: 5.58s
824:	learn: 0.5175172	total: 26.1s	remaining: 5.54s
825:	learn: 0.5174640	total: 26.2s	remaining: 5.51s
826:	learn: 0.5174317	total: 26.2s	remaining: 5.48s
827:	learn: 0.5173775	total: 26.3s	remaining: 5.45s
828:	learn: 0.5173654	total: 26.3s	remaining: 5.42s
829:	learn: 0.5173105	total: 26.3s	remaining: 5.39s
830:	learn: 0.5172896	total: 26.4s	remaining: 5.36s
831:	learn: 0.5172558	total: 26.4s	remaining: 5.33s
832:	learn: 0.5172206	total: 26.4s	remaining: 5.3s
833:	learn: 0.5171971	total: 26.5s	remaining: 5.27s
834:	learn: 0.517

978:	learn: 0.5112625	total: 31.1s	remaining: 668ms
979:	learn: 0.5112407	total: 31.2s	remaining: 636ms
980:	learn: 0.5112177	total: 31.2s	remaining: 605ms
981:	learn: 0.5112149	total: 31.2s	remaining: 573ms
982:	learn: 0.5111739	total: 31.3s	remaining: 541ms
983:	learn: 0.5111098	total: 31.3s	remaining: 509ms
984:	learn: 0.5110644	total: 31.3s	remaining: 477ms
985:	learn: 0.5110291	total: 31.4s	remaining: 446ms
986:	learn: 0.5110039	total: 31.4s	remaining: 414ms
987:	learn: 0.5109854	total: 31.4s	remaining: 382ms
988:	learn: 0.5109575	total: 31.5s	remaining: 350ms
989:	learn: 0.5109216	total: 31.5s	remaining: 318ms
990:	learn: 0.5109015	total: 31.5s	remaining: 286ms
991:	learn: 0.5108294	total: 31.5s	remaining: 254ms
992:	learn: 0.5108051	total: 31.6s	remaining: 222ms
993:	learn: 0.5107639	total: 31.6s	remaining: 191ms
994:	learn: 0.5107204	total: 31.6s	remaining: 159ms
995:	learn: 0.5106391	total: 31.6s	remaining: 127ms
996:	learn: 0.5106174	total: 31.7s	remaining: 95.3ms
997:	learn:

In [744]:
model.get_params()

{'learning_rate': 0.099,
 'depth': 5,
 'l2_leaf_reg': 5,
 'loss_function': 'RMSE',
 'verbose': True}

In [746]:
y_pred_train = model.predict(column_transformer.transform(x_train))
y_pred_test = model.predict(column_transformer.transform(x_test))

print('train MAE: \t', mean_absolute_error(y_pred_train, y_train))

if submit == 2 or submit == 3:
    print('test MAE: \t', mean_absolute_error(y_pred_test, y_test))

train MAE: 	 0.2978317149790738


In [747]:
y_pred_test = y_scaler.inverse_transform(y_pred_test)

In [748]:
# Отправка в Kaggle
if submit == 1:
    result = sample_submission.copy()
    result['demand'] = y_pred_test
    result.to_csv('result_{0}.csv'.format('catboost'), index=False)
    print('Generating .csv file .... READY')

Generating .csv file .... READY


___
___
# `Описание`

При первых сабмитах мы пользовались только следующими исходными признаками:

- `period_dt`
- `location_id`
- `product_id`
- `PROMO1_FLAG`
- `PROMO2_FLAG`
- `PRICE_REGULAR`
- `PRICE_AFTER_DISC`
- `NUM_CONSULTANT`
- `AUTORIZATION_FLAG`

В последних 6-ти из них были заполнены пропуски. Признак `period_dt` был удален, а из него вычленили `day, month, year`.

После разделения признаков на числовые и категориальные, первые прошли через `StandardScaler`, а вторые - через `OneHotEncoder`.

Были испробованы разные ML-модели, запускались при параметрах по умолчанию. Лучший результат получился при использовании `CatBoost`.

In [8]:
from IPython.display import Image
#from IPython.core.display import HTML 
Image(
    url= "https://2.downloader.disk.yandex.ru/preview/aca23d4b4f2f83c0f2fef9c368f202e0e18a0f973c4278f04ab2db2014dc4c34/inf/hIglPA1UvTlbC50rheaquA0ZLPKDd3SEPbT7RSxrRDKeArNZWQeI62pB5hYCYgUYlMqYFqnIlz_AY-2XrXO_CA%3D%3D?uid=334642788&filename=%231.png&disposition=inline&hash=&limit=0&content_type=image%2Fpng&owner_uid=334642788&tknv=v2&size=1903x929",
    width=600,
    height=300)

___
Потом захотелось изучить, что лежит в таблицах `product` и `location`. Там по-видимому лежат признаки соответствующих товаров и магазинов.

Что было сделано как для таблицы `product`, так и для `location`:
- `срез таблицы по встречающимся в исходных данных товарам / магазинам (уменьшили число строк)`


- `просмотр числа уникальных значений в каждом столбце таблицы`


- `срез таблицы по вырожденным столбцам, которые либо повторяют друг друга, либо имеют слишком неравномерное распределение уникальных значений с точки зрения частоты их встречаемости`

При использовании новых полученных признаков (они категориальные, разумеется)  получились неоднозначеные результаты. Добавление признаков из `location` только ухудшало качество, а признаки из `product` - улучшали.

Также была предпринята попытка срезать признаки по критерию их важности. Во всех тех сабмитах (на картинке ниже), где это использовалось - качество ухудшалось.

In [10]:
from IPython.display import Image
#from IPython.core.display import HTML 
Image(
    url= "https://2.downloader.disk.yandex.ru/preview/1316877a7a76ec5ff0ef1d1964e0d172e447dfaa7f161090a245affa28494067/inf/b6WL01SB6o30ii5u6MHLFQ0ZLPKDd3SEPbT7RSxrRDKAM2cWwif7xd_X_kLLxONM9Ai8i6z99-I84clhHRbYPg%3D%3D?uid=334642788&filename=%232.png&disposition=inline&hash=&limit=0&content_type=image%2Fpng&owner_uid=334642788&tknv=v2&size=941x919",
    width=600,
    height=300)

___
Были испробованы `lag-features`, но они не улучшили качество, а наоборот.

In [13]:
from IPython.display import Image
#from IPython.core.display import HTML 
Image(
    url= "https://4.downloader.disk.yandex.ru/preview/b32fa7b51446e052517ddaeaa3527c4c4c807599d16a458d45a0ee74215bf5b8/inf/I5ztsqC3-8U_DCz5KdOsvQ0ZLPKDd3SEPbT7RSxrRDLl-aprY7dgAIApRZQFGk0elZ6MpGK2lLasC9MLZLSM0w%3D%3D?uid=334642788&filename=%233.png&disposition=inline&hash=&limit=0&content_type=image%2Fpng&owner_uid=334642788&tknv=v2&size=880x919",
    width=600,
    height=300)

___
Далее были попытки настроить параметры `CatBoost`, чтобы улучшить свой лучший результат. Этого удалось добиться уменьшив допустимую глубину дереревьев с 6-ти, которая была установлена по умолчанию, до 5-ти.

Изменение метрики (с `RMSE` на `MAE`) и увеличение числа итераций не дали положительного результата.

In [14]:
from IPython.display import Image
#from IPython.core.display import HTML 
Image(
    url= "https://2.downloader.disk.yandex.ru/preview/af1915403166ea76799d292cd74f3e95d49e3de9289f62065e99f33bb72c1226/inf/tY3Bh0Bv87_AAy2aHJvkjg0ZLPKDd3SEPbT7RSxrRDKn4jJPo7jmZy2_JRSuXgIROsJNbcm0zkxESfOIo9oRjg%3D%3D?uid=334642788&filename=%234.png&disposition=inline&hash=&limit=0&content_type=image%2Fpng&owner_uid=334642788&tknv=v2&size=880x919",
    width=600,
    height=300)

___
___